In [2]:
%load_ext autoreload
%autoreload 2

In [32]:
import model
from data_utils.loader import Loader
from model.deconfounder import Deconfounder
from pyro.optim import Adam

In [161]:
data_path = "data/ohe_movies.csv"
loader = Loader(data_path)

In [163]:
X_train, X_test, y_train, y_test = loader.get_train_test()
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

torch.Size([2544, 129]) torch.Size([637, 129]) torch.Size([2544]) torch.Size([637])


In [221]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np


'''
scaler = StandardScaler()
y_train_t = scaler.fit_transform(y_train.numpy().reshape(-1, 1))
y_test_t = scaler.transform(y_test.numpy().reshape(-1, 1))
'''

'''
y_train_t = np.log1p(y_train.numpy())
y_test_t = np.log1p(y_test.numpy())
'''

lr = LinearRegression()
lr.fit(X_train.numpy(), y_train.numpy())
y_pred = lr.predict(X_test.numpy())
mse = mean_squared_error(y_test.numpy(), y_pred)

y_mean = np.zeros(y_test.shape) + y_train.numpy().mean()
mean_mse = mean_squared_error(y_test.numpy(), y_mean)
print(f'Linear regression MSE: {mse:.4f}, mean baseline mse: {mean_mse:.4f}')

#log_mse = mean_squared_error(np.log1p(y_test.numpy()), np.log1p(y_pred))
#log_mean_mse = mean_squared_error(np.log1p(y_test.numpy()), np.log1p(y_mean))

Linear regression MSE: 1.8751, mean baseline mse: 1.9978


In [271]:
# linear conf
step1_opt = Adam({"lr": 0.0005})
step2_opt = Adam({"lr": 0.005})
# seed def = 3493204
deconfounder = Deconfounder(step1_opt, step2_opt, 
                            seed=5323,
                            step1_iters=4000, step2_iters=1500)

In [272]:
step1_params, step2_params = deconfounder.train(X_train, y_train)


 Training Z marginal and W parameter marginal...
[iteration 0001] loss: 376.1324
[iteration 0101] loss: 359.4549
[iteration 0201] loss: 338.7086
[iteration 0301] loss: 328.9077
[iteration 0401] loss: 314.5475
[iteration 0501] loss: 302.1743
[iteration 0601] loss: 287.8564
[iteration 0701] loss: 277.7061
[iteration 0801] loss: 266.3705
[iteration 0901] loss: 257.6144
[iteration 1001] loss: 247.6543
[iteration 1101] loss: 227.7157
[iteration 1201] loss: 213.3058
[iteration 1301] loss: 191.6192
[iteration 1401] loss: 158.5139
[iteration 1501] loss: 129.1670
[iteration 1601] loss: 95.9907
[iteration 1701] loss: 71.5628
[iteration 1801] loss: 53.1820
[iteration 1901] loss: 41.1869
[iteration 2001] loss: 35.7594
[iteration 2101] loss: 29.6489
[iteration 2201] loss: 27.7824
[iteration 2301] loss: 24.2861
[iteration 2401] loss: 22.9345
[iteration 2501] loss: 20.7526
[iteration 2601] loss: 20.8910
[iteration 2701] loss: 20.3350
[iteration 2801] loss: 20.0229
[iteration 2901] loss: 19.6403
[ite

In [279]:
#y_train_pred = deconfounder.do_predict(X_train)

y_test_pred, test_params = deconfounder.do_predict(X_test, num_samples=5000)
mean_squared_error(y_test_pred, y_test.numpy())

3.571259097311846

In [281]:
y_cond_pred, test_params = deconfounder.cond_predict(X_test, num_samples=5000)
mean_squared_error(y_cond_pred, y_test.numpy())

3.6083607922924124

# Region top actors comp

In [311]:
params = deconfounder.step2_params
actor_weights = params['weight_mean0'][:-50]
actor_weights

tensor([1.8182, 1.6285, 1.5604, 1.7617, 1.7346, 1.7701, 1.4960, 1.5053, 1.6079,
        1.7688, 1.4529, 1.3123, 1.5816, 1.4644, 1.3396, 1.6510, 1.6191, 1.6058,
        1.6220, 1.5604, 1.5907, 1.5962, 1.6871, 1.7202, 1.3986, 1.6934, 1.4617,
        1.3779, 1.4652, 1.1988, 1.1442, 1.6474, 1.4420, 1.6459, 1.6691, 1.4636,
        1.4873, 1.6070, 1.7141, 1.6042, 1.7807, 1.3410, 1.4711, 1.6033, 1.4159,
        1.6093, 1.6258, 1.6035, 1.7553, 1.8049, 1.5283, 1.7332, 1.7465, 1.4748,
        1.5671, 1.7713, 1.5871, 1.5560, 1.4548, 1.4841, 1.5508, 1.6885, 1.6681,
        1.4200, 1.6070, 1.5168, 1.4997, 1.6552, 1.5184, 1.6859, 1.6013, 1.5187,
        1.7189, 1.3657, 1.4830, 1.4036, 1.4509, 1.5208, 1.5160, 1.4138, 1.5976,
        1.6348, 1.5446, 1.3147, 1.5126, 1.6687, 1.6992, 1.3947, 1.6287, 1.5731,
        1.5393, 1.6992, 1.4589, 1.4735, 1.5179, 1.6099, 1.3243, 1.6860, 1.5757,
        1.8483, 1.6292, 1.4032, 1.5979, 1.4567, 1.6150, 1.8282, 1.6312, 1.6876,
        1.7191, 1.5725, 1.7013, 1.5100, 

In [290]:
actors = loader.data_X.columns[:-1]
actors

Index(['Adam Sandler', 'Alec Baldwin', 'Amy Adams', 'Angelina Jolie',
       'Anna Faris', 'Anne Hathaway', 'Anthony Hopkins', 'Anthony Mackie',
       'Antonio Banderas', 'Arnold Schwarzenegger',
       ...
       'Sylvester Stallone', 'Tom Cruise', 'Tom Hanks', 'Tom Wilkinson',
       'Tommy Lee Jones', 'Vince Vaughn', 'Will Ferrell', 'Will Smith',
       'Willem Dafoe', 'Woody Harrelson'],
      dtype='object', length=129)

In [292]:
import pandas as pd


actors_frame = pd.DataFrame(data=actor_weights.detach().numpy(), 
                            index=actors, 
                            columns=['weight'])

actors_frame

,weight
Adam Sandler,1.818185
Alec Baldwin,1.628481
Amy Adams,1.560416
Angelina Jolie,1.761667
Anna Faris,1.734576
...,...
Vince Vaughn,1.363815
Will Ferrell,1.483769
Will Smith,1.966060
Willem Dafoe,1.493105


In [295]:
actors_frame.sort_values(by='weight', ascending=False)

,weight
Will Smith,1.966060
Tom Cruise,1.852589
Ralph Fiennes,1.848259
Tom Hanks,1.847136
Robin Williams,1.828198
...,...
Ben Kingsley,1.312298
Steve Zahn,1.304400
Susan Sarandon,1.216737
Christopher Walken,1.198792


# Region Linear regression

In [297]:
import torch
import torch.nn as nn

import pyro
from pyro.distributions import Normal, Uniform, Delta
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.distributions.util import logsumexp
from pyro.infer import EmpiricalMarginal, TracePredictive
from pyro.infer.mcmc import MCMC, NUTS
import pyro.optim as optim
import pyro.poutine as poutine


# Lets define a new regression model
class RegressionModel(nn.Module):
    
    def __init__(self, p):
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)
#         self.factor = nn.Parameter(torch.tensor(1.0))
        
    def forward(self, x):

        return self.linear(x).reshape(X_train.shape[0])
    
    
p = 129  # number of actors
regression_model = RegressionModel(p)

In [298]:
def model(x_data, y_data):
    
    dims = x_data.shape[1]
    # weight, bias, and factor priors
    w_prior = Normal(torch.zeros(1, dims), torch.ones(1, dims)).to_event(1)
    b_prior = Normal(torch.tensor([[8.]]), torch.tensor([[1000.]])).to_event(1)
#     f_prior = Normal(0., 1.)

    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    scale = pyro.sample("linear.sigma", Uniform(0., 10.))
    
    lifted_module = pyro.random_module("module", regression_model, priors)
    lifted_reg_model = lifted_module()
#     with pyro.plate("map", len(x_data)):
    with pyro.plate("map", len(x_data)):
        prediction_mean = lifted_reg_model(x_data)
        pyro.sample("obs", Normal(prediction_mean, scale))

        return prediction_mean

In [299]:

from pyro.infer.autoguide import AutoDiagonalNormal
# initialize the autodiagonal with init_to_feasible instead of init_to_median
from pyro.infer.autoguide import init_to_feasible

In [300]:

optim = Adam({"lr": 0.03})
cond_model = pyro.condition(model, data = {"obs" : y_train.reshape(y_train.shape[0])})
guide = AutoDiagonalNormal(cond_model, init_loc_fn = init_to_feasible)
svi = SVI(cond_model, guide, optim, loss=Trace_ELBO(), num_samples=10000)

/home/ag3r/anaconda3/envs/causality/lib/python3.7/site-packages/pyro/infer/svi.py:53: FutureWarning: The `num_samples` argument to SVI is deprecated and will be removed in a future release. Use `pyro.infer.Predictive` class to draw samples from the posterior.
  'samples from the posterior.', FutureWarning)


In [302]:
pyro.set_rng_seed(101)
num_iterations = 1000
def train():
    pyro.clear_param_store()
    for j in range(num_iterations):
        loss = svi.step(X_train, y_train.reshape(1, y_train.shape[0]))
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss/len(X_train)))
            
train()

[iteration 0001] loss: 9.1608
[iteration 0101] loss: 4.6323
[iteration 0201] loss: 4.1594
[iteration 0301] loss: 3.8915
[iteration 0401] loss: 3.6893
[iteration 0501] loss: 3.5365
[iteration 0601] loss: 3.4035
[iteration 0701] loss: 3.2635
[iteration 0801] loss: 3.0610
[iteration 0901] loss: 2.3928


In [303]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name), pyro.param(name).shape)

AutoDiagonalNormal.loc Parameter containing:
tensor([-1.8485e+00,  8.5459e-01,  5.0851e-01,  3.1549e-01,  5.6810e-01,
         8.1622e-01,  9.2847e-01,  7.6569e-02, -3.8580e-02,  3.8488e-01,
         1.1272e+00, -9.9346e-02, -6.0358e-02,  3.4970e-01, -7.9366e-02,
        -1.6901e-01,  5.2637e-01,  5.3432e-01,  5.3909e-01,  2.9002e-01,
         2.0374e-01,  4.1247e-01,  5.1824e-01,  5.0375e-01,  6.5129e-01,
        -1.6786e-01,  5.9988e-01, -3.8645e-02, -1.1462e-01,  4.2468e-01,
        -4.5529e-01, -4.4450e-01,  6.8353e-01, -5.6986e-02,  5.2011e-01,
         7.4689e-01, -1.6481e-02,  1.5911e-01,  2.2712e-01,  7.6906e-01,
         3.3581e-01,  7.7821e-01, -2.8460e-01,  2.8065e-01,  2.5643e-01,
         4.1029e-02,  5.4705e-01,  6.6607e-01,  4.5912e-01,  5.3267e-01,
         7.7372e-01,  2.9395e-01,  6.4536e-01,  7.1141e-01,  5.3548e-02,
         3.0119e-01,  6.7707e-01,  3.2084e-01,  2.8665e-01, -7.4225e-02,
        -5.0868e-02,  4.5973e-01,  7.7304e-01,  6.7649e-01,  1.4523e-01,
      

In [304]:
linreg_params = {}
for name, value in pyro.get_param_store().items():
    linreg_params[name] = pyro.param(name)

In [306]:
linreg_mean = linreg_params['AutoDiagonalNormal.loc'][:-2]
linreg_mean.shape

torch.Size([129])

In [307]:
import pandas as pd


actors_frame = pd.DataFrame(data=linreg_mean.detach().numpy(), 
                            index=actors, 
                            columns=['weight'])

actors_frame

,weight
Adam Sandler,-1.848469
Alec Baldwin,0.854593
Amy Adams,0.508507
Angelina Jolie,0.315485
Anna Faris,0.568104
...,...
Vince Vaughn,0.588407
Will Ferrell,-0.386592
Will Smith,0.139761
Willem Dafoe,1.353994


In [309]:
actors_frame.sort_values(by='weight', ascending=False)

,weight
Willem Dafoe,1.353994
Russell Crowe,1.230186
Richard Jenkins,1.154806
Ben Affleck,1.127194
Tom Hanks,1.082718
...,...
Sylvester Stallone,-0.427047
Danny Glover,-0.444503
Colin Farrell,-0.455286
Mark Ruffalo,-0.530481
